## Practical 10 Text processing with LSTM

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

import numpy as np

%matplotlib inline

In [39]:
train_df= pd.read_csv('taska.txt',sep='\t',header=None,names=['id','text','class'])
train_df['text'] = train_df['text'].fillna("something").values
train_df['class'] = np.where(train_df['class']=='OFF', 1, 0)


In [40]:
t1=pd.read_csv('testset-levela.tsv',sep='\t')
t1.head()
t2=pd.read_csv('labels-levela.csv',sep=',',header=None,names=['id','class'])
t2.head()
test_df=pd.merge(t1,t2,how='inner',on=['id'])
test_df['class'] = np.where(test_df['class']=='OFF', 1, 0)

test_df.head()

,id,tweet,class
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,1
1,27014,"#ConstitutionDay is revered by Conservatives, ...",0
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0
3,13876,#Watching #Boomer getting the news that she is...,0
4,60133,#NoPasaran: Unity demo to oppose the far-right...,1


In [41]:
train_df['class'].value_counts()
#test_df['class'].value_counts()/



0    8840
1    4400
Name: class, dtype: int64

In [42]:
from sklearn.model_selection import train_test_split
# create training and testing vars
train, valid= train_test_split(train_df, test_size=0.2, random_state=1)

In [43]:
print(train.groupby('class').count()) 
print(valid.groupby('class').count())

         id  text
class            
0      7081  7081
1      3511  3511
         id  text
class            
0      1759  1759
1       889   889


In [44]:
test_df.head()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 860 entries, 0 to 859
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      860 non-null    int64 
 1   tweet   860 non-null    object
 2   class   860 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 26.9+ KB


In [45]:
X_train = train['text']
X_valid = valid['text']
y_train=train['class']
y_valid=valid['class']
X_test = test_df['tweet'].fillna("something").values
y_test=test_df['class']
y_test=test_df['class']

In [46]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip

--2020-11-22 13:41:47--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5828358084 (5.4G) [application/zip]
Saving to: ‘crawl-300d-2M-subword.zip.1’

ip.1                  2%[                    ] 154.52M  23.5MB/s    eta 4m 12s ^C


In [47]:
#!unzip glove*.zip
!unzip crawl-300d-2M-subword.zip


Archive:  crawl-300d-2M-subword.zip
replace crawl-300d-2M-subword.vec? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace crawl-300d-2M-subword.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [48]:
embed_size = 300

In [49]:
#embedding_path='glove.6B.300d.txt'
embedding_path='crawl-300d-2M-subword.vec'

embeddings_index = {}
with open(embedding_path,encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs


In [50]:
len(embeddings_index)

2000000

In [51]:
embeddings_index['dhrumin']

array([-4.000e-02, -3.250e-02,  6.260e-02, -7.100e-03,  6.350e-02,
       -9.060e-02,  2.400e-02, -8.090e-02,  5.300e-03,  6.200e-03,
        5.100e-03,  3.970e-02, -4.530e-02,  5.900e-03,  2.790e-02,
        4.930e-02,  4.490e-02,  2.460e-02,  2.610e-02,  1.600e-02,
       -2.700e-03,  3.110e-02,  7.360e-02, -1.580e-02, -1.080e-02,
        8.290e-02, -1.020e-01,  3.230e-02, -1.820e-02,  3.900e-03,
       -5.180e-02,  3.330e-02,  3.280e-02, -1.227e-01,  7.830e-02,
       -4.790e-02,  8.980e-02,  7.390e-02, -2.590e-02, -1.560e-02,
       -4.240e-02, -5.280e-02,  9.600e-03, -7.740e-02,  4.360e-02,
        3.330e-02, -2.270e-02, -8.300e-02, -9.330e-02, -3.890e-02,
       -5.680e-02,  7.170e-02, -3.900e-03,  1.998e-01, -4.220e-02,
       -3.690e-02,  8.100e-03, -8.490e-02, -1.230e-02, -1.530e-02,
       -4.000e-03, -9.630e-02, -7.030e-02, -6.710e-02, -4.200e-03,
        4.900e-03,  7.300e-03, -9.750e-02, -1.700e-03, -1.054e-01,
        1.799e-01, -5.150e-02, -3.970e-02, -1.420e-02, -7.620e

In [52]:
max_len = 30
max_words=10000

tok = Tokenizer(num_words=max_words)

tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)


In [53]:
X_train[1]
sequences_matrix[2]

array([   1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,   82, 1411,    4,    7, 1090], dtype=int32)

In [54]:
X_train[77]

"@USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER Don't forget @USER &amp; Democrat backed #Antifa"

In [55]:
word_index = tok.word_index
#prepare embedding matrix
num_words = min(max_words, len(word_index) + 1)
import numpy as np


In [56]:
len(word_index)
word_index

{'user': 1,
 'the': 2,
 'is': 3,
 'to': 4,
 'a': 5,
 'and': 6,
 'you': 7,
 'of': 8,
 'are': 9,
 'i': 10,
 'he': 11,
 'that': 12,
 'in': 13,
 'for': 14,
 'she': 15,
 'url': 16,
 'it': 17,
 'this': 18,
 'on': 19,
 'not': 20,
 'they': 21,
 'with': 22,
 'have': 23,
 'be': 24,
 'liberals': 25,
 'gun': 26,
 'so': 27,
 'all': 28,
 'control': 29,
 'antifa': 30,
 'your': 31,
 'like': 32,
 'what': 33,
 'as': 34,
 'but': 35,
 'just': 36,
 'maga': 37,
 'her': 38,
 'we': 39,
 'about': 40,
 'was': 41,
 'conservatives': 42,
 'if': 43,
 'who': 44,
 'will': 45,
 'do': 46,
 'people': 47,
 'no': 48,
 'my': 49,
 'his': 50,
 'by': 51,
 'at': 52,
 'from': 53,
 'has': 54,
 'or': 55,
 'how': 56,
 'an': 57,
 'amp': 58,
 'up': 59,
 'their': 60,
 'me': 61,
 'out': 62,
 'get': 63,
 'one': 64,
 'more': 65,
 'know': 66,
 'trump': 67,
 'why': 68,
 'can': 69,
 'when': 70,
 'because': 71,
 'them': 72,
 'think': 73,
 'him': 74,
 'now': 75,
 'would': 76,
 "don't": 77,
 'there': 78,
 'should': 79,
 'right': 80,
 'our': 8

In [57]:
word_index = tok.word_index
#prepare embedding matrix
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
embedding_matrix.shape

(10000, 300)

In [58]:
embedding_matrix[1]

array([-0.0312    , -0.1725    ,  0.109     ,  0.0666    ,  0.1096    ,
       -0.0567    ,  0.1462    ,  0.008     ,  0.0539    ,  0.0407    ,
       -0.0155    , -0.0214    , -0.0045    ,  0.0686    ,  0.0539    ,
       -0.0596    ,  0.0071    ,  0.0969    ,  0.0454    ,  0.13169999,
       -0.0024    , -0.0184    , -0.0505    ,  0.0121    , -0.0061    ,
        0.1119    , -0.1138    ,  0.0483    , -0.0561    ,  0.1045    ,
       -0.0279    , -0.1285    , -0.0924    ,  0.0052    ,  0.0944    ,
       -0.0528    , -0.0238    , -0.1247    ,  0.0891    , -0.0802    ,
        0.0578    , -0.25529999,  0.0024    , -0.0758    , -0.0809    ,
        0.1027    , -0.029     ,  0.0626    , -0.0688    , -0.0611    ,
        0.0291    ,  0.0455    ,  0.13249999,  0.13869999, -0.0631    ,
        0.0439    ,  0.0362    ,  0.036     , -0.0834    ,  0.0378    ,
        0.0943    , -0.0451    , -0.1032    , -0.0144    , -0.0164    ,
        0.0098    , -0.01      , -0.0774    ,  0.0886    , -0.01

In [59]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(num_words,300,input_length=max_len,weights=[embedding_matrix])(inputs)
    #layer = Embedding(num_words,300,input_length=max_len)(inputs)

    #layer = Embedding(max_words,300,input_length=max_len)(inputs)

    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [60]:
from keras.callbacks import ModelCheckpoint

model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 30)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 30, 300)           3000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                93440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                

In [61]:
from keras.callbacks import ModelCheckpoint

file_path = "lstm_model_glove300_.{epoch:02d}-{val_acc:.8f}.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                              save_best_only = False, mode = "min")



In [62]:
valid_sequences = tok.texts_to_sequences(X_valid)
valid_sequences_matrix = sequence.pad_sequences(valid_sequences,maxlen=max_len)


test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)


In [63]:
valid_sequences_matrix[1]

array([ 289,  133,  254, 4384,    8,  959, 1565,   68,    9,    2,   42,
         27,  939, 1132,   43,    7,  390,   61,   83, 6997,   34,   21,
       1127,  547,   55,    9,   21,   36,   12, 1132], dtype=int32)

In [64]:
#model.fit(sequences_matrix,y_train, batch_size=128, epochs=5, validation_data=(valid_sequences_matrix, y_valid), callbacks = [check_point])

model.fit(sequences_matrix,y_train, batch_size=128, epochs=2, validation_data=(valid_sequences_matrix, y_valid))

Epoch 1/2
83/83 [==============================] - 3s 38ms/step - loss: 0.5910 - accuracy: 0.6935 - val_loss: 0.5394 - val_accuracy: 0.7379
Epoch 2/2
83/83 [==============================] - 3s 33ms/step - loss: 0.4151 - accuracy: 0.8185 - val_loss: 0.5185 - val_accuracy: 0.7606


In [65]:
Y_valid=y_valid
from sklearn import metrics
import keras
y_pred_prob=model.predict(valid_sequences_matrix)
#y_pred_class = model.
y_pred_class = (y_pred_prob >= 0.5).astype(np.int)
print(metrics.confusion_matrix(Y_valid, y_pred_class))
print (metrics.precision_score(Y_valid, y_pred_class,average= 'weighted'))
print(metrics.recall_score(Y_valid, y_pred_class,average= 'weighted'))
print (metrics.f1_score(Y_valid, y_pred_class, average= 'weighted'))
#print (metrics.f1_score(y_test, y_pred_class))
print(metrics.classification_report(Y_valid, y_pred_class))

[[1458  301]
 [ 333  556]]
0.7585764887497342
0.7605740181268882
0.7594590686180253
              precision    recall  f1-score   support

           0       0.81      0.83      0.82      1759
           1       0.65      0.63      0.64       889

    accuracy                           0.76      2648
   macro avg       0.73      0.73      0.73      2648
weighted avg       0.76      0.76      0.76      2648



In [66]:
ty_pred_prob=model.predict(test_sequences_matrix)
ty_pred_class = (ty_pred_prob >= 0.5).astype(np.int)

print(metrics.confusion_matrix(y_test, ty_pred_class))
print (metrics.precision_score(y_test, ty_pred_class,average= 'weighted'))
print(metrics.recall_score(y_test, ty_pred_class,average= 'weighted'))
print (metrics.f1_score(y_test, ty_pred_class, average= 'weighted'))
print (metrics.f1_score(y_test, ty_pred_class))
print(metrics.accuracy_score(y_test,ty_pred_class))
print(metrics.classification_report(y_test, ty_pred_class,target_names=['0','1']))
test_df['pred_taska']=ty_pred_class
#test_df.head()
#test_df[test_df['pred_taska']==1].info()


[[526  94]
 [ 92 148]]
0.7842780521817448
0.7837209302325582
0.7839949416161045
0.6141078838174274
0.7837209302325582
              precision    recall  f1-score   support

           0       0.85      0.85      0.85       620
           1       0.61      0.62      0.61       240

    accuracy                           0.78       860
   macro avg       0.73      0.73      0.73       860
weighted avg       0.78      0.78      0.78       860



In [67]:
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [68]:
# fix random seed for reproducibility
seed = 7
import numpy
numpy.random.seed(seed)
model_cnn_02 = Sequential()
e = Embedding(10000, 300, input_length=30,weights=[embedding_matrix])
#e = Embedding(10000, 300, input_length=1000)

model_cnn_02.add(e)
model_cnn_02.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_02.add(GlobalMaxPooling1D())
model_cnn_02.add(Dense(256, activation='relu'))
model_cnn_02.add(Dense(1, activation='sigmoid'))
#file_path = "CNNbest_model.hdf5"
file_path = "model-taska/CNNbest_model_en_fasttext300.{epoch:02d}-{val_acc:.8f}.hdf5"

check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                              save_best_only = False, mode = "min")
#ra_val = RocAucEvaluation(validation_data=(X_valid, Y_valid), interval = 1)
early_stop = EarlyStopping(monitor = "val_loss", min_delta=0.0001)
model_cnn_02.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [69]:
model_cnn_02.fit(sequences_matrix,y_train, batch_size=128, epochs=2, validation_data=(valid_sequences_matrix, y_valid))

Epoch 1/2
83/83 [==============================] - 3s 32ms/step - loss: 0.5791 - accuracy: 0.7014 - val_loss: 0.5045 - val_accuracy: 0.7727
Epoch 2/2
83/83 [==============================] - 3s 31ms/step - loss: 0.4098 - accuracy: 0.8184 - val_loss: 0.4814 - val_accuracy: 0.7776


In [70]:
from sklearn import metrics
import keras
model=model_cnn_02
from sklearn import metrics
import keras
y_pred_prob=model.predict(valid_sequences_matrix)
#y_pred_class = model.
y_pred_class = (y_pred_prob >= 0.5).astype(np.int)
print(metrics.confusion_matrix(Y_valid, y_pred_class))
print (metrics.precision_score(Y_valid, y_pred_class,average= 'weighted'))
print(metrics.recall_score(Y_valid, y_pred_class,average= 'weighted'))
print (metrics.f1_score(Y_valid, y_pred_class, average= 'weighted'))
#print (metrics.f1_score(y_test, y_pred_class))
print(metrics.classification_report(Y_valid, y_pred_class))



[[1567  192]
 [ 397  492]]
0.7714858502226487
0.7775679758308157
0.7691977364132122
              precision    recall  f1-score   support

           0       0.80      0.89      0.84      1759
           1       0.72      0.55      0.63       889

    accuracy                           0.78      2648
   macro avg       0.76      0.72      0.73      2648
weighted avg       0.77      0.78      0.77      2648



In [71]:
model=model_cnn_02

ty_pred_prob=model.predict(test_sequences_matrix)
ty_pred_class = (ty_pred_prob >= 0.5).astype(np.int)

print(metrics.confusion_matrix(y_test, ty_pred_class))
print (metrics.precision_score(y_test, ty_pred_class,average= 'weighted'))
print(metrics.recall_score(y_test, ty_pred_class,average= 'weighted'))
print (metrics.f1_score(y_test, ty_pred_class, average= 'weighted'))
print (metrics.f1_score(y_test, ty_pred_class))
print(metrics.accuracy_score(y_test,ty_pred_class))
print(metrics.classification_report(y_test, ty_pred_class,target_names=['0','1']))
test_df['pred_taska']=ty_pred_class
#test_df.head()
#test_df[test_df['pred_taska']==1].info()


[[572  48]
 [105 135]]
0.8149879686063124
0.8220930232558139
0.8140156727529788
0.6382978723404255
0.8220930232558139
              precision    recall  f1-score   support

           0       0.84      0.92      0.88       620
           1       0.74      0.56      0.64       240

    accuracy                           0.82       860
   macro avg       0.79      0.74      0.76       860
weighted avg       0.81      0.82      0.81       860

